In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
from sklearn.metrics import roc_curve,auc,confusion_matrix
import seaborn as sns

In [50]:
df_1 = pd.read_csv('dataForProjects/playerdata/transfermarkt_fbref_201718.csv',sep=';',index_col=[0])
df_2 = pd.read_csv('dataForProjects/playerdata/transfermarkt_fbref_201819.csv',sep=';',index_col=[0])
df_3 = pd.read_csv('dataForProjects/playerdata/transfermarkt_fbref_201920.csv',sep=';',index_col=[0])

df = pd.concat([df_2,df_1,df_3])

C:\Users\anshs\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (395) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [51]:
df[df['squad']=='Manchester City']

,player,nationality,position,squad,age,birth_year,value,height,position2,foot,...,Pts/G,xG,xGA,xGDiff,xGDiff/90,Attendance,CL,WinCL,CLBestScorer,Season
22.0,Sergio Agüero,ar ARG,FW,Manchester City,30.0,1988.0,65000000.0,173.0,Forward - Centre-Forward,right,...,2.58,82.0,25.6,56.4,1.49,54130,1.0,0.0,0.0,201819#
576.0,Danilo,br BRA,DF,Manchester City,27.0,1991.0,20000000.0,184.0,Defender - Right-Back,right,...,2.58,82.0,25.6,56.4,1.49,54130,1.0,0.0,0.0,201819#
588.0,Kevin De Bruyne,be BEL,MF,Manchester City,27.0,1991.0,130000000.0,181.0,Midfielder - Attacking Midfield,right,...,2.58,82.0,25.6,56.4,1.49,54130,1.0,0.0,0.0,201819#
609.0,Fabian Delph,eng ENG,DF,Manchester City,28.0,1989.0,15000000.0,174.0,Midfielder - Central Midfield,left,...,2.58,82.0,25.6,56.4,1.49,54130,1.0,0.0,0.0,201819#
728.0,Ederson,br BRA,GK,Manchester City,24.0,1993.0,70000000.0,188.0,Goalkeeper,left,...,2.58,82.0,25.6,56.4,1.49,54130,1.0,0.0,0.0,201819#
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2316.0,David Silva,es ESP,MF,Manchester City,33.0,1986.0,12000000.0,170.0,Midfielder - Attacking Midfield,left,...,2.13,93.0,34.7,58.3,1.53,37097,1.0,0.0,0.0,201920#
2399.0,Raheem Sterling,eng ENG,FW,Manchester City,24.0,1994.0,128000000.0,170.0,Forward - Left Winger,right,...,2.13,93.0,34.7,58.3,1.53,37097,1.0,0.0,0.0,201920#
2405.0,John Stones,eng ENG,DF,Manchester City,25.0,1994.0,36000000.0,188.0,Defender - Centre-Back,right,...,2.13,93.0,34.7,58.3,1.53,37097,1.0,0.0,0.0,201920#
2637.0,Kyle Walker,eng ENG,DF,Manchester City,29.0,1990.0,40000000.0,183.0,Defender - Right-Back,right,...,2.13,93.0,34.7,58.3,1.53,37097,1.0,0.0,0.0,201920#


In [52]:
for j in range(len([i for i in df.columns])):
    print ('\item '+[i for i in df.columns][j])

\item player
\item nationality
\item position
\item squad
\item age
\item birth_year
\item value
\item height
\item position2
\item foot
\item league
\item games
\item games_starts
\item minutes
\item goals
\item assists
\item pens_made
\item pens_att
\item cards_yellow
\item cards_red
\item goals_per90
\item assists_per90
\item goals_assists_per90
\item goals_pens_per90
\item goals_assists_pens_per90
\item xg
\item npxg
\item xa
\item xg_per90
\item xa_per90
\item xg_xa_per90
\item npxg_per90
\item npxg_xa_per90
\item minutes_90s
\item shots_total
\item shots_on_target
\item shots_free_kicks
\item shots_on_target_pct
\item shots_total_per90
\item shots_on_target_per90
\item goals_per_shot
\item goals_per_shot_on_target
\item npxg_per_shot
\item xg_net
\item npxg_net
\item passes_completed
\item passes
\item passes_pct
\item passes_total_distance
\item passes_progressive_distance
\item passes_completed_short
\item passes_short
\item passes_pct_short
\item passes_completed_medium
\item 

In [66]:
player_tackles = df[['player','squad','Season','minutes','tackles','tackles_won','tackles_def_3rd','tackles_mid_3rd','tackles_att_3rd','blocks','interceptions']]

In [67]:
player_tackles.head()

,player,squad,Season,minutes,tackles,tackles_won,tackles_def_3rd,tackles_mid_3rd,tackles_att_3rd,blocks,interceptions
207.0,Borja Bastón,Alavés,201819#,1439.0,15.0,9.0,1.0,12.0,2.0,18.0,1.0
363.0,Burgui,Alavés,201819#,555.0,7.0,5.0,4.0,2.0,1.0,5.0,0.0
399.0,Jonathan Calleri,Alavés,201819#,2869.0,24.0,19.0,7.0,11.0,6.0,38.0,8.0
704.0,Rubén Duarte,Alavés,201819#,2772.0,72.0,40.0,38.0,23.0,11.0,90.0,34.0
738.0,Rodrigo Ely,Alavés,201819#,430.0,6.0,4.0,4.0,2.0,0.0,16.0,4.0


In [55]:
player_tackles['90s'] = player_tackles['minutes'].apply(lambda x: x/90)

C:\Users\anshs\AppData\Local\Temp/ipykernel_18360/192822924.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_tackles['90s'] = player_tackles['minutes'].apply(lambda x: x/90)


In [57]:
# convert stats to per 90 stats
player_tackles[['tackles','tackles_won','tackles_def_3rd','tackles_mid_3rd','tackles_att_3rd','blocks','interceptions']] = player_tackles[['tackles','tackles_won','tackles_def_3rd','tackles_mid_3rd','tackles_att_3rd','blocks','interceptions']].div(player_tackles['90s'], axis=0)

C:\Users\anshs\anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [58]:
player_tackles.head()

,player,squad,Season,minutes,tackles,tackles_won,tackles_def_3rd,tackles_mid_3rd,tackles_att_3rd,blocks,interceptions,90s
207.0,Borja Bastón,Alavés,201819#,1439.0,0.938151,0.562891,0.062543,0.750521,0.125087,1.125782,0.062543,15.988889
363.0,Burgui,Alavés,201819#,555.0,1.135135,0.810811,0.648649,0.324324,0.162162,0.810811,0.000000,6.166667
399.0,Jonathan Calleri,Alavés,201819#,2869.0,0.752876,0.596026,0.219589,0.345068,0.188219,1.192053,0.250959,31.877778
704.0,Rubén Duarte,Alavés,201819#,2772.0,2.337662,1.298701,1.233766,0.746753,0.357143,2.922078,1.103896,30.800000
738.0,Rodrigo Ely,Alavés,201819#,430.0,1.255814,0.837209,0.837209,0.418605,0.000000,3.348837,0.837209,4.777778


In [87]:
player_tackles[(player_tackles['squad']=="Real Madrid")&(player_tackles['Season']=="201819#")]['90s'].sum()/11

30.02828282828283

In [68]:
team_df_1 = pd.read_csv('dataForProjects/playerdata/team_data_1819.csv',index_col=[0],encoding="iso-8859-1")
team_df_1['Season'] = '201819#'
team_df_2 = pd.read_csv('dataForProjects/playerdata/team_data_1920.csv',index_col=[0])
team_df_2['Season'] = '201920#'
team_df = pd.concat([team_df_1,team_df_2])
team_df.rename(columns = {'Squad':'squad'}, inplace = True)


In [69]:
team_df.head()

,squad,Country,LgRk,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Attendance,Top Team Scorer,Goalkeeper,Season
Rk,,,,,,,,,,,,,,,,,,,,
1,Manchester City,eng ENG,1,38,32,2,4,95,23,72,98,2.58,84.3,24.7,59.6,1.57,54130,Sergio Agüero - 21,Ederson,201819#
2,Liverpool,eng ENG,2,38,30,7,1,89,22,67,97,2.55,73.7,28.8,44.9,1.18,52983,Sadio Mané Mohamed Salah - 22,Alisson,201819#
3,Paris S-G,fr FRA,1,38,29,4,5,105,35,70,91,2.39,88.9,37.6,51.3,1.35,46911,Kylian Mbappé - 33,Alphonse Areola,201819#
4,Juventus,it ITA,1,38,28,6,4,70,30,40,90,2.37,60.7,35.8,25.0,0.66,37799,Cristiano Ronaldo - 21,Wojciech Szczesny,201819#
5,Bayern Munich,de GER,1,34,24,6,4,88,32,56,78,2.29,85.6,27.2,58.4,1.72,75000,Robert Lewandowski - 22,Manuel Neuer,201819#


In [71]:
player_tackles = player_tackles.groupby(['squad','Season'], as_index=False).agg({"tackles": "sum",
                                                               "tackles_won": "sum",
                                                               "tackles_att_3rd": "sum",
                                                               "tackles_def_3rd": "sum",
                                                               "tackles_mid_3rd": "sum",
                                                                "blocks": "sum",
                                                                "interceptions": "sum"})

In [72]:
final_df = player_tackles.merge(team_df, how='inner',on=['squad','Season'])
final_df


,squad,Season,tackles,tackles_won,tackles_att_3rd,tackles_def_3rd,tackles_mid_3rd,blocks,interceptions,Country,...,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Attendance,Top Team Scorer,Goalkeeper
0,Alavés,201819#,450.0,250.0,53.0,237.0,160.0,469.0,246.0,es ESP,...,-11,50,1.32,41.1,53.2,-12.1,-0.32,17295,Jonathan Calleri - 9,Fernando Pacheco
1,Amiens,201819#,614.0,426.0,74.0,294.0,246.0,547.0,228.0,fr FRA,...,-21,38,1.00,35.4,49.8,-14.4,-0.38,11043,Moussa Konaté - 7,Régis Gurtner
2,Amiens,201920#,483.0,321.0,44.0,229.0,210.0,460.0,268.0,fr FRA,...,-19,23,0.82,27.4,39.3,-11.9,-0.43,11616,Sehrou Guirassy - 9,Régis Gurtner
3,Angers,201819#,635.0,432.0,75.0,325.0,235.0,549.0,354.0,fr FRA,...,-5,46,1.21,50.7,47.9,2.8,0.07,11189,Stéphane Bahoken - 11,Ludovic Butelle
4,Angers,201920#,496.0,319.0,67.0,227.0,202.0,398.0,236.0,fr FRA,...,-5,39,1.39,29.3,28.4,0.9,0.03,9825,Rachid Alioui - 6,Ludovic Butelle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,West Ham,201920#,719.0,470.0,67.0,399.0,253.0,632.0,290.0,eng ENG,...,-13,39,1.03,48.6,63.8,-15.3,-0.40,44155,Michail Antonio - 10,Łukasz Fabiański
185,Wolfsburg,201819#,489.0,330.0,58.0,249.0,182.0,442.0,254.0,de GER,...,12,55,1.62,47.6,48.3,-0.7,-0.02,24479,Wout Weghorst - 17,Koen Casteels
186,Wolfsburg,201920#,653.0,401.0,78.0,316.0,259.0,590.0,281.0,de GER,...,2,49,1.44,52.8,42.9,9.9,0.29,18621,Wout Weghorst - 16,Koen Casteels
187,Wolves,201819#,605.0,359.0,49.0,328.0,228.0,590.0,306.0,eng ENG,...,1,57,1.50,52.1,42.1,10.1,0.26,31030,Raúl Jiménez - 13,Rui Patrício


In [73]:
final_df[['tackles_att_3rd','tackles_def_3rd','tackles_mid_3rd','blocks','interceptions','tackles']] = final_df[['tackles_att_3rd','tackles_def_3rd','tackles_mid_3rd','blocks','interceptions','tackles']].div(final_df['MP'],axis=0)

In [74]:
final_df[['tackles_att_3rd','tackles_def_3rd','tackles_mid_3rd']] = final_df[['tackles_att_3rd','tackles_def_3rd','tackles_mid_3rd']].div(final_df['tackles'],axis=0)

In [75]:
final_df['goals_per_90'] = final_df['GA'].div(final_df['MP'],axis=0)

In [79]:
final_df=final_df[final_df['tackles']>8]
final_df

,squad,Season,tackles,tackles_won,tackles_att_3rd,tackles_def_3rd,tackles_mid_3rd,blocks,interceptions,Country,...,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Attendance,Top Team Scorer,Goalkeeper,goals_per_90
0,Alavés,201819#,11.842105,250.0,0.117778,0.526667,0.355556,12.342105,6.473684,es ESP,...,50,1.32,41.1,53.2,-12.1,-0.32,17295,Jonathan Calleri - 9,Fernando Pacheco,1.315789
1,Amiens,201819#,16.157895,426.0,0.120521,0.478827,0.400651,14.394737,6.000000,fr FRA,...,38,1.00,35.4,49.8,-14.4,-0.38,11043,Moussa Konaté - 7,Régis Gurtner,1.368421
2,Amiens,201920#,17.250000,321.0,0.091097,0.474120,0.434783,16.428571,9.571429,fr FRA,...,23,0.82,27.4,39.3,-11.9,-0.43,11616,Sehrou Guirassy - 9,Régis Gurtner,1.785714
3,Angers,201819#,16.710526,432.0,0.118110,0.511811,0.370079,14.447368,9.315789,fr FRA,...,46,1.21,50.7,47.9,2.8,0.07,11189,Stéphane Bahoken - 11,Ludovic Butelle,1.289474
4,Angers,201920#,17.714286,319.0,0.135081,0.457661,0.407258,14.214286,8.428571,fr FRA,...,39,1.39,29.3,28.4,0.9,0.03,9825,Rachid Alioui - 6,Ludovic Butelle,1.178571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,West Ham,201920#,18.921053,470.0,0.093185,0.554937,0.351878,16.631579,7.631579,eng ENG,...,39,1.03,48.6,63.8,-15.3,-0.40,44155,Michail Antonio - 10,Łukasz Fabiański,1.631579
185,Wolfsburg,201819#,14.382353,330.0,0.118609,0.509202,0.372188,13.000000,7.470588,de GER,...,55,1.62,47.6,48.3,-0.7,-0.02,24479,Wout Weghorst - 17,Koen Casteels,1.470588
186,Wolfsburg,201920#,19.205882,401.0,0.119449,0.483920,0.396631,17.352941,8.264706,de GER,...,49,1.44,52.8,42.9,9.9,0.29,18621,Wout Weghorst - 16,Koen Casteels,1.352941
187,Wolves,201819#,15.921053,359.0,0.080992,0.542149,0.376860,15.526316,8.052632,eng ENG,...,57,1.50,52.1,42.1,10.1,0.26,31030,Raúl Jiménez - 13,Rui Patrício,1.210526


In [80]:
final_df.to_csv(r'dataForProjects/playerdata/team_def_data_1820_final.csv')

In [86]:
final_df[(final_df['tackles']==final_df['tackles'].min())]

,squad,Season,tackles,tackles_won,tackles_att_3rd,tackles_def_3rd,tackles_mid_3rd,blocks,interceptions,Country,...,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Attendance,Top Team Scorer,Goalkeeper,goals_per_90
101,Mainz 05,201819#,9.529412,175.0,0.135802,0.429012,0.435185,10.764706,5.588235,de GER,...,43,1.26,51.2,59.2,-8.0,-0.24,26246,Jean-Philippe Mateta - 14,Florian Müller,1.676471


In [87]:
final_df[(final_df['tackles']==final_df['tackles'].max())]

,squad,Season,tackles,tackles_won,tackles_att_3rd,tackles_def_3rd,tackles_mid_3rd,blocks,interceptions,Country,...,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Attendance,Top Team Scorer,Goalkeeper,goals_per_90
110,Metz,201920#,22.964286,405.0,0.091757,0.567652,0.340591,17.321429,9.25,fr FRA,...,34,1.21,27.1,38.1,-11.0,-0.39,16583,Habib Diallo - 12,Alexandre Oukidja,1.25


In [88]:
final_df[(final_df['tackles_def_3rd']==final_df['tackles_def_3rd'].min())]

,squad,Season,tackles,tackles_won,tackles_att_3rd,tackles_def_3rd,tackles_mid_3rd,blocks,interceptions,Country,...,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Attendance,Top Team Scorer,Goalkeeper,goals_per_90
18,Bayern Munich,201920#,17.382353,396.0,0.182741,0.35533,0.461929,14.441176,6.882353,de GER,...,82,2.41,89.5,37.9,51.6,1.52,57353,Robert Lewandowski - 34,Manuel Neuer,0.941176


In [89]:
final_df[(final_df['tackles_def_3rd']==final_df['tackles_def_3rd'].max())]

,squad,Season,tackles,tackles_won,tackles_att_3rd,tackles_def_3rd,tackles_mid_3rd,blocks,interceptions,Country,...,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Attendance,Top Team Scorer,Goalkeeper,goals_per_90
122,Newcastle Utd,201920#,17.631579,392.0,0.091045,0.613433,0.295522,16.763158,7.631579,eng ENG,...,44,1.16,35.5,65.6,-30.1,-0.79,35551,Jonjo Shelvey - 6,Martin Dúbravka,1.526316


In [90]:
final_df[(final_df['blocks']==final_df['blocks'].min())]

,squad,Season,tackles,tackles_won,tackles_att_3rd,tackles_def_3rd,tackles_mid_3rd,blocks,interceptions,Country,...,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Attendance,Top Team Scorer,Goalkeeper,goals_per_90
119,Napoli,201819#,10.342105,255.0,0.167939,0.473282,0.358779,9.263158,5.052632,it ITA,...,79,2.08,61.8,38.2,23.5,0.62,29003,Arkadiusz Milik - 17,David Ospina,0.947368


In [91]:
final_df[(final_df['blocks']==final_df['blocks'].max())]

,squad,Season,tackles,tackles_won,tackles_att_3rd,tackles_def_3rd,tackles_mid_3rd,blocks,interceptions,Country,...,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Attendance,Top Team Scorer,Goalkeeper,goals_per_90
129,Paderborn 07,201920#,20.117647,403.0,0.087719,0.559942,0.352339,19.852941,7.676471,de GER,...,20,0.59,39.2,67.3,-28.1,-0.83,11038,Sven Michel Dennis Srbeny... - 5,Leopold Zingerle,2.176471


In [92]:
final_df[(final_df['interceptions']==final_df['interceptions'].min())]

,squad,Season,tackles,tackles_won,tackles_att_3rd,tackles_def_3rd,tackles_mid_3rd,blocks,interceptions,Country,...,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Attendance,Top Team Scorer,Goalkeeper,goals_per_90
17,Bayern Munich,201819#,10.647059,239.0,0.174033,0.403315,0.422652,10.764706,4.882353,de GER,...,78,2.29,85.6,27.2,58.4,1.72,75000,Robert Lewandowski - 22,Manuel Neuer,0.941176


In [93]:
final_df[(final_df['interceptions']==final_df['interceptions'].max())]

,squad,Season,tackles,tackles_won,tackles_att_3rd,tackles_def_3rd,tackles_mid_3rd,blocks,interceptions,Country,...,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Attendance,Top Team Scorer,Goalkeeper,goals_per_90
121,Newcastle Utd,201819#,18.157895,405.0,0.094203,0.530435,0.375362,18.763158,10.894737,eng ENG,...,45,1.18,39.1,53.6,-14.5,-0.38,51121,Ayoze Pérez - 12,Martin Dúbravka,1.263158
